# **Vision and Perception**
```
Name: Antonio 
Last Name: Costini
Mat: 59295
```
``` 
Name: Rosario
Last Name: Catena
Mat: 59290
 ```



# TRAFFIC LIGHT AND SIGNAL DETECTION USING DARKNET

## **CONFIGURATION**

**In this section we will proceed to configure our Darknet network.**

**All the configuration files are contained in the drive, so we will proceed to mount Google Drive on the Colab session,**







Mounting DRIVE... specifies the root folder.




In [ ]:
from google.colab import drive

print("mounting DRIVE...")
drive.mount('/content/gdrive')
root_folder = 'VisionePercezione_AntonioCostini_RosarioCatena' #@param {type:"string"}
!ln -s /content/gdrive/My\ Drive/$root_folder /my_drive

**Now we will proceed to clone the [repository](https://github.com/AlexeyAB/darknet), we're going to set some configuration parameters such as:**

*   **OPENCV** *To compile with opencv,*
*   **GPU** *To speedup on GPU,*
*   **CUDNN** *To speedup on GPU,*
*   **CUDNN_HALF** *To further speedup 3 x times,*

**The next step is to compile.**

Proceed with the compilation by selecting the desired configuration parameters.

In [ ]:
!git clone https://github.com/AlexeyAB/darknet
%cd darknet

OPENCV = True #@param {type:"boolean"}
GPU = True #@param {type:"boolean"}
CUDNN = True #@param {type:"boolean"}
CUDNN_HALF = True #@param {type:"boolean"}

print("setting properties...")
if OPENCV:
  print("activating OPENCV...")
  !sed -i 's/OPENCV=0/OPENCV=1/' Makefile

if GPU:
  print("engines CUDA...")
  !/usr/local/cuda/bin/nvcc --version
  
  print("activating GPU...")
  !sed -i 's/GPU=0/GPU=1/' Makefile

if CUDNN:
  print("activating CUDNN...")
  !sed -i 's/CUDNN=0/CUDNN=1/' Makefile

if CUDNN_HALF:
  print("activating CUDNN_HALF...")
  !sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

print("making...")
!make

print("FINISH!")

**To proceed we will load the dataset in order to use it for training.**

Enter the name of the folder containing the dataset (the dataset by convention must be named "obj.zip")




In [ ]:
dataset_folder = 'dataset' #@param {type:"string"}

print("loading dataset...")
!cp /my_drive/$dataset_folder/obj.zip ../

print("unziping dataset...")
!unzip ../obj.zip -d data

Enter the name of the folder containing the configuration files, provide:
- yolo-obj.cfg
- obj.names
- obj.data



In [ ]:
configuration_folder = 'configuration_files' #@param {type:"string"}

print("loading yolo-obj.cfg...")
!cp /my_drive/$configuration_folder/yolo-obj.cfg ./cfg

print("loading obj.names...")
!cp /my_drive/$configuration_folder/obj.names ./data

print("loading obj.data...")
!cp /my_drive/$configuration_folder/obj.data ./data

**Darknet needs a .txt file for training which must contain the paths for each image, so i wrote a script that does this.
Load and run it.**







Enter the name of the folder containing scripts and the name of the script that generates the .txt file.


In [ ]:
script_folder = 'py_scripts' #@param {type:"string"}

script_file = 'generate_train.py' #@param {type:"string"}

print("loading script...")
!cp /my_drive/$script_folder/$script_file ./
print("performing script...")
!python $script_file

Enter the name of the folder containing weights and the name of the pre-trained weights file.


In [ ]:
weights_folder = 'backup' #@param {type:"string"}

pre_trained_weights_file = 'yolov4.conv.137' #@param {type:"string"}

print("loading pre_trained weights...")
!cp /my_drive/$weights_folder/$pre_trained_weights_file ./

**After having correctly configured the network we can proceed with the next section.**

## **TRAINING**


**In this section we will train the network.**

Choose whether to show the mAP (mean average precisions) calculation every 100 iterations and whether you want to start or resume the training.




In [ ]:
train_using_mAP = False #@param {type:"boolean"}

option = 'START TRAINING FROM BEGINNING' #@param ["START TRAINING FROM BEGINNING", "RESUME TRAINING"]

if option == 'START TRAINING FROM BEGINNING':
  if train_using_mAP:
    !./darknet detector train data/obj.data cfg/yolo-obj.cfg $pre_trained_weights_file -dont_show -map
  else:
    co
else:
  if train_using_mAP:
    !./darknet detector train data/obj.data cfg/yolo-obj.cfg /my_drive/$weights_folder/yolo-obj_last.weights -dont_show -map
  else:
    !./darknet detector train data/obj.data cfg/yolo-obj.cfg /my_drive/$weights_folder/yolo-obj_last.weights -dont_show


 ** The file yolo-obj_last.weights will be saved to the backup folder for each 100 iterations, while file yolo-obj_xxxx.weights will be saved to the backup folder for each 1000 iterations**

Save the graph in the Drive, set the range of iterations made.




In [ ]:
initial_iteration_number = 0 #@param {type:"slider", min:0, max:10000, step:100}
final_iteration_number = 0 #@param {type:"slider", min:0, max:10000, step:100}

chart_name = "mAP-chart_iter:{}-{}.png".format(initial_iteration_number, final_iteration_number)

print("saving chart...")

!cp chart.png /my_drive/charts/$chart_name


## **METRICS**

Enter the name of the weights on which to calculate the metrics

In [ ]:
weights_name = 'yolo-obj_last.weights' #@param {type:"string"}

!cp /my_drive/backup/$weights_name ./

!./darknet detector map data/obj.data cfg/yolo-obj.cfg $weights_name

## **DETECTION**##



**When the training is complete you can test on an image the final weights file by editing the my_dataset.cfg file**

In [ ]:
image_test_folder = 'test_images' #@param {type:"string"}
input_name_image = 'image_xxxx.JPG' #@param {type:"string"}
#weights_type = 'yolo last' #@param ["yolo best", "yolo last"] 
weights_type = 'yolo-obj_last.weights' #@param {type:"string"}
predictions_folder = 'predictions' #@param {type:"string"}
output_name_prediction = 'prediction_image_xxxx.JPG' #@param {type:"string"}

print("detecting...")
!./darknet detector test data/obj.data cfg/yolo-obj.cfg /my_drive/backup/$weights_type -dont_show /my_drive/$image_test_folder/$input_name_image -out_filename predictions.jpg
# Show the result using the helper imgShow()
#imShow('predictions.jpg')
print("copying prediction in Drive...")
!cp predictions.jpg /my_drive/$predictions_folder/$output_name_prediction

**In this section we will perform object detection on the videos and save the results in the Drive**

In [ ]:
video_test_folder = 'test_videos' #@param {type:"string"}
input_name = 'video_xxxx.MOV' #@param {type:"string"}
#weights_type = 'yolo last' #@param ["yolo best", "yolo last"]
weights_type = 'yolo-obj_last.weights' #@param {type:"string"}
predictions_folder = 'predictions' #@param {type:"string"}
output_name = 'prediction_video_xxxx' #@param {type:"string"}
prediction_version =   1

prediction_name = "{}_prediction_version:{}.avi".format(output_name, prediction_version)

print("detecting...")
!./darknet detector demo data/obj.data cfg/yolo-obj.cfg /my_drive/backup/$weights_type -dont_show /my_drive/$video_test_folder/$input_name -thresh .7 -i 0 -out_filename prediction.avi

print("copying prediction in Drive...")
!cp prediction.avi /my_drive/$predictions_folder/$prediction_name
